In [32]:
import pandas as pd
import matplotlib.pyplot as plt
import json

In [21]:
## inspecionando o json

with open('../data/01_raw/data.json') as f:
    data = json.load(f)

data

[{'CreateDate': '2021-05-24T20:21:34.79',
  'EmissionDate': '2021-05-24T00:00:00',
  'Discount': 0.0,
  'NFeNumber': 501,
  'NFeID': 1,
  'ItemList': [{'ProductName': 'Rice', 'Value': 35.55, 'Quantity': 2},
   {'ProductName': 'Flour', 'Value': 11.55, 'Quantity': 5},
   {'ProductName': 'Bean', 'Value': 27.15, 'Quantity': 7}]},
 {'CreateDate': '2021-05-24T20:21:34.79',
  'EmissionDate': '2021-05-24T00:00:00',
  'Discount': 0.0,
  'NFeNumber': 502,
  'NFeID': 2,
  'ItemList': [{'ProductName': 'Tomate', 'Value': 12.25, 'Quantity': 10},
   {'ProductName': 'Pasta', 'Value': 7.55, 'Quantity': 5}]},
 {'CreateDate': '2021-05-24T20:21:34.79',
  'EmissionDate': '2021-05-24T00:00:00',
  'Discount': 0.0,
  'NFeNumber': 503,
  'NFeID': 3,
  'ItemList': [{'ProductName': 'Beer', 'Value': 9.0, 'Quantity': 6},
   {'ProductName': 'French fries', 'Value': 10.99, 'Quantity': 2},
   {'ProductName': 'Ice cream', 'Value': 27.15, 'Quantity': 1}]}]

In [28]:
## normalizando todo o dataset e criando uma dataset único
sales_full = pd.json_normalize(data, meta=['CreateDate','EmissionDate',
                                               'Discount','NFeNumber','NFeID'],
                                  record_path=['ItemList'])

## normalizando todo o dataset, mas mantendo somente os metadados
sales_metadata = pd.json_normalize(data, meta=['CreateDate','EmissionDate',
                                               'Discount','NFeNumber','NFeID'])
sales_metadata.drop(['ItemList'], axis=1, inplace=True)

## noramlizando o dataset, mas mantendo somente os items comprados e o metadado de ID
## da NFe para possível cruzamento.
products_sold = pd.json_normalize(data, record_path=['ItemList'], 
                       meta=['NFeID'])

display(sales_full.head())
display(sales_metadata.head())
display(products_sold.head())

,ProductName,Value,Quantity,CreateDate,EmissionDate,Discount,NFeNumber,NFeID
0,Rice,35.55,2,2021-05-24T20:21:34.79,2021-05-24T00:00:00,0.0,501,1
1,Flour,11.55,5,2021-05-24T20:21:34.79,2021-05-24T00:00:00,0.0,501,1
2,Bean,27.15,7,2021-05-24T20:21:34.79,2021-05-24T00:00:00,0.0,501,1
3,Tomate,12.25,10,2021-05-24T20:21:34.79,2021-05-24T00:00:00,0.0,502,2
4,Pasta,7.55,5,2021-05-24T20:21:34.79,2021-05-24T00:00:00,0.0,502,2


,CreateDate,EmissionDate,Discount,NFeNumber,NFeID
0,2021-05-24T20:21:34.79,2021-05-24T00:00:00,0.0,501,1
1,2021-05-24T20:21:34.79,2021-05-24T00:00:00,0.0,502,2
2,2021-05-24T20:21:34.79,2021-05-24T00:00:00,0.0,503,3


,ProductName,Value,Quantity,NFeID
0,Rice,35.55,2,1
1,Flour,11.55,5,1
2,Bean,27.15,7,1
3,Tomate,12.25,10,2
4,Pasta,7.55,5,2


In [42]:
## cruzando dados

merged = (
    products_sold
    .merge(sales_metadata, 
           left_on='NFeID', 
           right_on='NFeID', 
           how='left')
)

merged.head()

,ProductName,Value,Quantity,NFeID,CreateDate,EmissionDate,Discount,NFeNumber
0,Rice,35.55,2,1,2021-05-24T20:21:34.79,2021-05-24T00:00:00,0.0,501
1,Flour,11.55,5,1,2021-05-24T20:21:34.79,2021-05-24T00:00:00,0.0,501
2,Bean,27.15,7,1,2021-05-24T20:21:34.79,2021-05-24T00:00:00,0.0,501
3,Tomate,12.25,10,2,2021-05-24T20:21:34.79,2021-05-24T00:00:00,0.0,502
4,Pasta,7.55,5,2,2021-05-24T20:21:34.79,2021-05-24T00:00:00,0.0,502
